In [1]:
import pandas as pd

In [2]:
# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)

# Euro 2024 Tournament Data
df = pd.read_csv("2024.csv")

In [3]:
## new features: add base camps

In [4]:
# convert data types to datetime objects
df['date'] = pd.to_datetime(df['date'])
df['date_time'] = pd.to_datetime(df['date_time'])

In [5]:
df.sort_values('date_time', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df_stadium = df[["home_team", "away_team", "stadium_city", "stadium_latitude", "stadium_longitude", "date_time"]]

In [7]:
df_stadium.head()

,home_team,away_team,stadium_city,stadium_latitude,stadium_longitude,date_time
0,Germany,Scotland,Munich,48.218794,11.624731,2024-06-14 19:00:00+00:00
1,Hungary,Switzerland,Cologne,50.933506,6.874994,2024-06-15 13:00:00+00:00
2,Spain,Croatia,Berlin,52.514714,13.239397,2024-06-15 16:00:00+00:00
3,Italy,Albania,Dortmund,51.492589,7.451772,2024-06-15 19:00:00+00:00
4,Poland,Netherlands,Hamburg,53.587156,9.898553,2024-06-16 13:00:00+00:00
